In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("models/research/")
sys.path.append("models/research/object_detection")

from object_detection.utils import ops as utils_ops

In [ ]:
# This is needed to display the images.
%matplotlib inline


In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [ ]:
# What model to download.# What  

MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = os.path.join(os.getcwd(),'models/research/object_detection',MODEL_NAME ,'frozen_inference_graph.pb')
print(os.path)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(os.getcwd(),'models/research/object_detection','data', 'mscoco_label_map.pbtxt')
print(os.path)

NUM_CLASSES = 90

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
import cv2
cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach

detection_graph.as_default()
sess = tf.Session(graph=detection_graph)
while True:
    ret, image_np = cap.read()
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
#         print image_np_expanded
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
#         print image_tensor

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')

    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
      [boxes, scores, classes, num_detections],
      feed_dict={image_tensor: image_np_expanded})
    #         print num_detections
    boxes = np.squeeze(boxes)
    classes= np.squeeze(classes)
    scores = np.squeeze(scores)

    #         print classes
    max_boxes_to_draw = 20
    display_str = ''
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[i]>0.5:
            box = tuple(boxes[i].tolist())
            if classes[i] in category_index.keys():
                class_name = category_index[classes[i]]['name']
            else:
                class_name = 'N/A'
            display_str = str(class_name)
        print "Dedicated Object"
        print display_str
        print box

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,                         
        use_normalized_coordinates=True,
        line_thickness=8)

    cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
    if cv2.waitKey(25)& 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break

In [ ]:
cap.release()